In [11]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
import re
import os
import zipfile

In [2]:
import sys
import time

# author imbr (https://stackoverflow.com/a/34482761)
def progressbar(it, prefix="", size=60, out=sys.stdout):
    count = len(it)
    start = time.time()
    def show(j):
        x = int(size*j/count)
        remaining = ((time.time() - start) / j) * (count - j)
        
        mins, sec = divmod(remaining, 60)
        time_str = f"{int(mins):02}:{sec:05.2f}"
        
        print(f"{prefix}[{u'█'*x}{('.'*(size-x))}] {j}/{count} Est wait {time_str}", end='\r', file=out, flush=True)
        
    for i, item in enumerate(it):
        yield item
        show(i+1)
    print("\n", flush=True, file=out)

In [3]:
def get_gene_list(path:str, column:str, base=None):

    data = pd.read_csv(path)

    if base:
        name_list = list(data.loc[pd.isnull(data.loc[:,base]), column].unique())
    else:
        name_list = list(data.loc[:,column].unique())
    
    return name_list

In [4]:
def find_gene_code(gene:str):
    
    route = 'https://www.ncbi.nlm.nih.gov/gene/?term=Homo+sapiens+' + gene
    page = requests.get(route)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    div = soup.find_all("div", id="primary")
    if div:
        # best case: correct match
        for d in div:
            link = d.find("a")['href']
            return link.split('/')[-1]
        
    div = soup.find_all("div", id="gene-tabular-docsum")
    if div:
        # second case: article list
        for d in div:
            link = d.find("a")
            ponctuation = r"(?:-| |\.|_)"
            aux_gene = re.sub(ponctuation, "", gene.lower())
            span = link.find("span")
            if span:
                aux_link = re.sub(ponctuation, "", span.contents[0].lower())
            else:
                aux_link = re.sub(ponctuation, "", link.contents[0].lower())
                
            if aux_link in aux_gene:
                return link['href'].split('/')[-1]
            else:
                return None
            
    span = soup.find_all("span", class_="geneid")
    if span:
        # third case: directly to download page (risky)
        for s in span:
            gene_id = re.sub(r"[^a-zA-Z\d ]", "", s.contents[0]).split()[2]
            return gene_id if re.match(r"^\d+$", gene_id) else None

In [5]:
def download_gene(gene, dest_dir:str='./data/files'):
        
    gene_id = find_gene_code(gene)
    
    if gene_id:
        response = requests.get(
            f'https://api.ncbi.nlm.nih.gov/datasets/v2alpha/gene/id/{gene_id}/download?include_annotation_type=FASTA_RNA'
        )
        if response.status_code==200:
            path = dest_dir + '/' + gene + '.zip'
            with open(path, 'ba+') as f:
                f.write(response.content)
    else:     
        with open('./data/not_found.txt', 'a+') as f:
            f.write(gene+'\n')

In [140]:
def get_gene_sequence(path, download_path, data, complete:bool=True, save:bool=False, dest_path='./data/RNAs'):
    downloaded_gene = [i[:-4] for i in os.listdir(download_path)]
    
    count = 0
    
    for gene in downloaded_gene:
        file = download_path+'/'+gene+'.zip'
        file = zipfile.ZipFile(file, 'r')
        ponctuation = r"(?:-| |\.|_)"
        
        if "hsa" in gene:
            aux_gene = gene[4:]
            aux_gene = re.sub(r"(?:-|\.|_)", " ", aux_gene).lower().split()[0]
        else:
            aux_gene = re.sub(r"(?:-|\.|_)", " ", gene).lower().split()[0]

        if 'ncbi_dataset/data/rna.fna' in file.namelist():
            file = file.read('ncbi_dataset/data/rna.fna')
            if complete:
                
                # get labels
                l = [line for line in file.decode("utf-8").split('\n') if line and not re.match(r'^[CTAG]+', line)]
                
                # check if names match
                isin = False
                for label in l:
                    aux_label = re.sub(ponctuation, "",label.split('[')[0]).lower().strip()
                    if aux_gene in aux_label or \
                       label.split()[1] in re.sub(ponctuation, "", gene).lower():
                        isin = True
                        break
                
                # change table value
                if isin:
                    data.loc[data['Nomenclatura']==gene, 'Sequenciamento Genômico'] = file.decode("utf-8")
                else:
                    count += 1
            else:
                lines = file.decode("utf-8").split('\n')
                
                # get labels
                lables = [i for i in range(len(lines)) if lines[i] and (not re.match(r'^[CTAG]+', lines[i]))]
                
                selected_gene = ''
                
                # look for gene sequences that match with gene name
                for i, l in enumerate(lables):
                    aux_label = re.sub(ponctuation, "",lines[l].split('[')[0]).lower()
                    if aux_gene in aux_label or \
                       lines[l].split()[1] in re.sub(ponctuation, "", gene).lower():
                        if i == len(lables)-1:
                            selected_gene += '\n'.join(lines[l:])
                        else:
                            selected_gene += '\n'.join(lines[l:lables[i+1]])

                if selected_gene:
                    data.loc[data['Nomenclatura']==gene, 'Sequenciamento Genômico'] = selected_gene
    
    print(count)
    if save:
        import datetime
        time = datetime.datetime.now().strftime("%H-%M_%d-%m-%Y")
        mode = '_complete_' if complete else '_selected_'
        dest_path = dest_path + mode + time + '.csv'
        data.to_csv(dest_path, index=False)
    
    return data
        

In [6]:
#find_gene_code('https://www.ncbi.nlm.nih.gov/gene/?term=Homo+sapiens+AOC4P')
#find_gene_code('https://www.ncbi.nlm.nih.gov/gene/?term=Homo+sapiens+hsa-mir-34a')
find_gene_code('https://www.ncbi.nlm.nih.gov/gene/?term=Homo+sapiens+NR_029373')

'641518'

In [12]:
path = 'data/RNAs.csv'
column = 'Nomenclatura'
base = 'Sequenciamento Genômico'
dest_path = './data/files'

continue_download = False
new_download = True

In [8]:
last_checked = None
downloaded = [i[:-4] for i in os.listdir(dest_path)]

if continue_download:
    with open('data/not_found.txt') as f:
        last_checked = f.read().split('\n')[-2]

In [9]:
gene_list = [i for i in get_gene_list(path, column, base) if i not in downloaded]
len(gene_list)

3467

In [10]:
if last_checked:
    start = gene_list.index(last_checked)
else:
    start = 0

for i in progressbar(range(start, len(gene_list)), "Genes: ", 60):
    download_gene(gene_list[i])

Genes: [████████████████████████████████████████████████████████████] 3467/3467 Est wait 00:00.009



In [141]:
complete = True
data = pd.read_csv(path)

In [142]:
get_gene_sequence(path, dest_path, data, complete=complete, save=True)

3855


,Sequenciamento Genômico,Nomenclatura,Tipos de RNAs,Tipos de Cancer,Método de Validação,Amostra,Padrão de disfunção,Informação Associada,Pubmed ID
0,>NR_002773.1 AOC4P [organism=Homo sapiens] [Ge...,AOC4P,LncRNA,colon cancer,ChIP//Immunoblot//RIP//RNAi//qRT-PCR,NaN,Expression,UPAT and UHRF1 may be promising molecular targ...,26768845.0
1,GGCTCACCGCAACCTCCACCGTCCTGAGTTAAAGTGATTCTCCTGT...,BACE1-AS,LncRNA,colon cancer,qPCR,"cell line (SNU-C4R, SNU-C5R)",Regulation [down-regulated],"We selected three lncRNAs,snaR,BACE1AS,and PRA...",25078450.0
2,AGTTAGTGCTGGGAAACAGTGCTAAGAAGGATACAGTGGCTAGAAG...,BCAR4,LncRNA,colon cancer,qRT-PCR,colon cancer tissues and paired normal tissues,Regulation [up-regulated],"In our research, we found that the expression ...",29190958.0
3,GGCCGGGCGCGGTGGCTCACGCCTGTAATCCCAGCTCTCAGGGAGG...,BCYRN1,LncRNA,colon cancer,qRT-PCR,colon cancer tissues and adjacent non-cancerou...,Regulation [up-regulated],"Here, we found that expression of BC200 was up...",29625226.0
4,TTTAAATCATACCAATTGAACCGAGCCTTGTAGAAACACTATCACC...,CCAT1,LncRNA,colon cancer,ISH//qPCR,colonic tissue,Regulation [up-regulated],CCAT1 is up-regulated across the colon adenoma...,23594791.0
...,...,...,...,...,...,...,...,...,...
16431,>NR_029970.1 MIR451A [organism=Homo sapiens] [...,hsa-mir-451,MiRNA,colorectal carcinoma,NaN,NaN,down,MicroRNA-451 Inhibits Growth of Human Colorect...,NaN
16432,>NR_030355.1 MIR625 [organism=Homo sapiens] [G...,hsa-mir-625-3p,MiRNA,colorectal carcinoma,NaN,NaN,up,MiR-625-3p promotes cell migration and invasio...,NaN
16433,>NR_030281.1 MIR92B [organism=Homo sapiens] [G...,hsa-mir-92b-3p,MiRNA,colorectal carcinoma,NaN,NaN,up,miR-92b-3p Promotes Colorectal Carcinoma Cell ...,NaN
16434,>NR_029510.1 MIR93 [organism=Homo sapiens] [Ge...,hsa-mir-93,MiRNA,colorectal carcinoma,NaN,NaN,down,MicroRNA-93 suppress colorectal cancer develop...,NaN


In [136]:
comp_file = 'RNAs_complete_13-42_05-02-2024.csv'
comp_file = 'RNAs_selected_13-58_05-02-2024.csv'
data = pd.read_csv(path)
new_data = pd.read_csv('./data/'+comp_file)

diff = data.loc[pd.isnull(data.loc[:, 'Sequenciamento Genômico'])].shape[0] - \
        new_data.loc[pd.isnull(new_data.loc[:, 'Sequenciamento Genômico'])].shape[0]

1162